In [1]:
#konlpy 설치
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 45.3 MB/s eta 0:00:00


In [2]:
!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=a4a7f6c536392e53b3bc5cd070de589e6ccf5b896646051ddd5a11303e9341ae
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [3]:
!pip install -U --no-cache-dir gdown --pre
!gdown --no-cookies --id 1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS
From (redirected): https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS&confirm=t&uuid=2afd7b29-bfac-4c2e-a722-ec2a2b0be100
To: /content/train_original.json
100% 800M/800M [00:14<00:00, 55.1MB/s]


In [4]:
from konlpy.tag import Hannanum, Okt, Kkma

In [5]:
from summa import keywords
from sklearn.metrics import jaccard_score
import gensim
from gensim.models.word2vec import Word2Vec
import re
import pandas as pd
from tqdm import tqdm, tqdm_pandas
import gensim
import pickle
tqdm_pandas(tqdm())
hannanum = Hannanum()
kkma = Kkma()
okt = Okt()

0it [00:00, ?it/s]


In [8]:
with open('ffinal_data.pkl', 'rb') as f :
    li = pickle.load(f)

In [6]:
stopwords = ['저자', '작가', '소개', '인기', '우리', '계기', '보유', '내용', '이야기']

In [7]:
def preprocess(string) :
    string = ' '.join(re.findall('[가-힣]+|[a-z]+', string.lower())) # 특수문자들 제거 및 모든 영어 소문자 변환
    string = okt.nouns(string)
    for ind, i in enumerate(string) :
        if i in stopwords :
            string[ind] = ''
    string = ' '.join(string)
    string = ' '.join(re.findall('\w{2,}', string.lower())) # 1글자인 단어들 삭제
    words_n = 5 # 뽑을 키워드 갯수

    if len(set(string.split())) < words_n :
        words_n = len(set(string.split())) # 단어 갯수가 적을 경우 뽑을 키워드 갯수를 낮춘다

    string = keywords.keywords(string, words = words_n, scores = True) # 키워드 뽑기

    word = []
    score = []
    for i, j in string :
        i = ''.join(i).split()
        word.append(i)
    word_list = []
    for i in word :
        for j in i :
            word_list.append(j)
    for i in word_list : # 단어들 1개씩
        for ind, j in enumerate(word) :
            if i in j :
                score.append([i, round(string[ind][1], 2)])
    if len(score) > 5 :
        score = score[:5]

    return score

In [9]:
def DicTrans(keyword) : # 키워드들을 딕셔너리 형태로 변환

    intro = [i[0] for i in keyword]

    dic = {}
    for ind, i in enumerate(intro) :
        dic[i] = keyword[ind][1]

    return dic

In [10]:
def JarccardSimilarity (input_key, data_key) : #
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key.keys())
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + (data_key[i] * 1.3)
    a = set.union(in_keys ,da_keys)
    return score / len(a)

In [11]:
def JarccardSimilarity_title (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key)
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + 1.3
    a = set.union(in_keys, da_keys)
    return score / len(a)

In [86]:
def Recommendation(input_text, data_set, Book_Count) : # 사용자가 넣은 문장을 전처리 하여, 데이터 셋과 유사도 분석 실행, 상위 n개 출력

    if type(input_text) != dict :
        input_text = preprocess(input_text)
        input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a : JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a : JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a : JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1 : -(Book_Count + 1) : -1] # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1 : -(Book_Count + 1) : -1]
    title_sim_list = title_score.sort_values()[-1 : -(Book_Count + 1) : -1]

    for title , t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]) :
        print('==================================================================================================================')
        print(f'{title}')
        print('==================================================================================================================')
        print('==================================================================================================================')
        for key, score in zip(t.keys(), t) :
            if score == 0 :
                break
            for i in ['상품명','저자', '소개글', '목차', '분야', '정가'] :
                print(f'{i} : {data_set.loc[key][i]}')
            print(f'유사도 : {score}')
            print('==================================================================================================================')

###실행 파일

In [87]:
# 책 리스트 불러오기
with open('ffinal_data.pkl', 'rb') as f:
    Book = pickle.load(f).reset_index(drop=True)
Your_Field = ''
loop = 1
# 토글
while loop:
    try:
        Book_Field = input("분야를 입력 하시겠습니까? (True(입력): 1, False(미입력): 0): ")
        Book_Field = int(Book_Field)  # 입력 값을 정수로 변환

        if Book_Field not in (0, 1):
            raise ValueError('1 또는 0을 입력하세요.')

        if Book_Field == 1:
            print(Book['분야'].unique())
            Your_Field = input("분야를 입력 해주세요: ")

            if Your_Field not in list(Book['분야'].unique()): # 분야 범위 필터링
                print(f'시스템을 종료합니다 분야를 확인해주세요, your_input: {Your_Field}')

            else:
                book_list = Book[Book['분야'] == Your_Field] # 분야에 따라 책 리스트를 필터링

        else:
            print('분야를 입력하지 않았습니다.', '전체 범위에서 추천을 수행합니다.')




        Book_Count = int(input("추천 받을 책 권수를 입력해주세요 (0 이상의 정수): "))

        Your_Text = input('과제를 입력 해주세요: ')

        if len(preprocess(Your_Text)) == 0 :
            Your_Text = {Your_Text : 1}

        if Your_Field :
            field_data = Book[Book['분야'] == Your_Field]
            not_field_data = Book[Book['분야'] != Your_Field]

            print('선택하신 분야 외의 도서 추천 리스트 입니다')

            Recommendation(Your_Text, field_data, Book_Count)

            print('선택하신 분야 외의 도서 추천 리스트 입니다')

            Recommendation(Your_Text, not_field_data, Book_Count)

        else :

            print('도서 추천 리스트 입니다')

            Recommendation(Your_Text, Book, Book_Count)

        loop = input("처음으로 돌아가시겠습니까? (예(입력): 1, 아니오(미입력): 0): ")

    except ValueError as e:
        print(e, f'your_input: {Book_Field}')
    except KeyboardInterrupt:
        print('\n사용자에 의해 프로그램이 종료되었습니다.')
        break


분야를 입력 하시겠습니까? (True(입력): 1, False(미입력): 0): 0
분야를 입력하지 않았습니다. 전체 범위에서 추천을 수행합니다.
추천 받을 책 권수를 입력해주세요 (0 이상의 정수): 5
과제를 입력 해주세요: 역사
도서 추천 리스트 입니다
소개글 기반 유사도
상품명 : 외우지 않고 통째로 이해하는 통세계사 2: 근대의 형성에서 현대까지
저자 : 김상훈
소개글 : 청소년부터 성인까지 모든 세대를 위한 살아있는 역사
외우지 않아도 통째로 머리에 남는 세계사 수업!
역사 공부를 힘들어 하거나 조금만 어려워져도 성적이 떨어지는 건 역사를 암기 과목으로만 접했기 때문이다. 역사를 달달 외워 얻은 지식은 책을 덮는 순간 사라져버린다. 중요한 건 맥락이다. 역사의 흐름을 이해할 수 있다면 더 이상 헷갈리거나 까먹지 않고 역사를 술술 말할 수 있게 된다. 이 책은 스토리텔링을 통해 역사의 맥을 잡게 하여 힘들게 외우지 않아도 통째로 머리에 남게 한다. 게다가 국내 역사서 중 유일하게 세계사와 한국사, 동양사와 서양사를 구분하지 않고 결합해서 설명하기 때문에 통합된 인류사라는 넓은 교양 지식을 쌓을 수 있다. 저자는 기자 특유의 간결하고 명쾌한 글솜씨로 맛깔나게 서술해 주는데, 청소년부터 성인까지 누구나 읽을 수 있도록 어려운 역사 용어는 전부 쉽게 풀었다. 그동안 역사를 외워야만 한다고 생각해 왔던 이들에게 이 책은 역사의 즐거움을 느끼게 해 주고 지식을 쌓아 약점이었던 역사가 아닌 자신의 강점이 되는 기회를 마련해 줄 것이다.
목차 : [2권] 근대의 형성에서 현대까지

11장 동서양 바다 열리다
1400~1500년 전후

12장 중세의 종말
1500~1600년 전후

13장 시민사회의 출범
1600~1700년 전후

14장 근대 혁명이 터지다
1700~1800년 전후

15장 자유와 혁명의 시대
1800~1850년 전후

16장 팽창하는 제국주의
1850~1900년 전후

17장 전쟁, 세계 파괴하다
1900~1950년 전후

18장 냉전과 화합
19

In [88]:
len('==================================================================================================================')

114